Download and import dataset using Torch Vision

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

# bool to set network use
use_conv_net = False

# Normalize 32x32 3 channels 0.5 mean and std
transform_method = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

if use_conv_net:
    batch_size = 32
else:
    batch_size = 32

dataset_classes = (
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
)

# Download and transform datasets
training_data = torchvision.datasets.CIFAR10(
    root='./CIFAR10',
    train=True, download=True,
    transform=transform_method
)

testing_data = torchvision.datasets.CIFAR10(
    root='./CIFAR10',
    train=False,
    download=True,
    transform=transform_method
)

# Create loaders for training and testing datasets
training_data_loader = DataLoader(
    training_data,
    batch_size=batch_size,
    # Shuffle training dataset
    shuffle=True
)

testing_data_loader = DataLoader(
    testing_data,
    batch_size=batch_size,
    shuffle=False
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./CIFAR10/cifar-10-python.tar.gz to ./CIFAR10
Files already downloaded and verified


Define Neural Network and CNN with layers

In [2]:
import torch.nn as nn
import torch.nn.functional as F


# Increase initial out channels if gpu is available
if torch.cuda.is_available():
    out_channels = 24
else:
    out_channels = 6

# Implement dropout in both models
dropout_rate = 0.2
in_hidden_layer = 128
out_hidden_layer = 64

# Neural network
class NeuralNet(nn.Module):
    def __init__(self, **kwargs):
        super(NeuralNet, self).__init__(**kwargs)
        # First linear input features defined as 3 channels 32x32 img size
        # 512 feature hidden layer
        self.linear1 = nn.Linear(in_features=3 * 32 * 32, out_features=128)
        self.linear2 = nn.Linear(in_features=in_hidden_layer, out_features=out_hidden_layer)
        self.linear3 = nn.Linear(in_features=out_hidden_layer, out_features=10)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input):
        output = torch.flatten(input, 1)
        output = self.linear1(output)
        output = F.relu(output)
        output = self.linear2(output)
        output = F.relu(output)
        output = self.dropout(output)
        output = self.linear3(output)
        return output

# CNN
class ConvNet(nn.Module):
    def __init__(self, **kwargs):
        super(ConvNet, self).__init__(**kwargs)
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=out_channels, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=out_channels,
                               out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input):
        output = self.pool(F.relu(self.conv1(input)))
        output = self.pool(F.relu(self.conv2(output)))
        output = torch.flatten(output, 1)
        output = F.relu(self.fc1(output))
        output = F.relu(self.fc2(output))
        output = self.dropout(output)
        output = self.fc3(output)
        return output

# Set torch to static seed
torch.random.manual_seed(1)
# Detect if gpu hardware is avaiable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if use_conv_net:
    net = ConvNet().to(device)
else:
    net = NeuralNet().to(device)

Define loss function and optimiser

In [3]:
import torch.optim as optim

# Default learning rate
learning_rate = 0.03

# Loss function
loss_function = nn.CrossEntropyLoss()

# Set optimiser to stochastic gradient descent with momentum
optimiser = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.7)

Train using Neural Net

In [4]:
from tqdm import tqdm

# Set amount of epochs
number_of_epochs = 5

for epoch in range(number_of_epochs):
    epoch_loss = 0.0
    with tqdm(training_data_loader, unit='batch') as tqdm_epoch:
        for i, data in enumerate(tqdm_epoch):
            tqdm_epoch.set_description(f"epoch {epoch + 1}")
            if torch.cuda.is_available():
                inputs, labels = data[0].to(device), data[1].to(device)
            else:
                inputs, labels = data
            # Set the gradient of all optimized tensors to zero.
            optimiser.zero_grad()

            outputs = net(inputs)
            loss = loss_function(outputs, labels)
            # Compute the gradient of current tensor
            loss.backward()
            # Perform single optimization step
            optimiser.step()
            epoch_loss += loss.item()
            tqdm_epoch.set_postfix(loss=(epoch_loss/tqdm_epoch.total))

# Save nn model
torch.save(net.state_dict(), './trained_net.pt')

epoch 5: 100%|██████████| 1563/1563 [00:34<00:00, 45.25batch/s, loss=1.38]


Test Accuarcy

In [5]:
correct_predictions = 0
# Don't calculate grad
with torch.no_grad():
    # Each test image
    for data in testing_data_loader:
        if torch.cuda.is_available():
            inputs, labels = data[0].to(device), data[1].to(device)
        else:
            inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        correct_predictions += (predicted == labels).sum().item()

if use_conv_net:
    print(
        f'CNN Accuracy: {(100 * correct_predictions / len(testing_data_loader.dataset))}%')
else:
    print(
        f'NN Accuracy: {(100 * correct_predictions / len(testing_data_loader.dataset))}%')

NN Accuracy: 48.25%
